### 파파고 API 사용법

In [25]:
import os
import sys
import urllib.request
import json
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import csv
from tqdm import tqdm
import datetime # 현재 시간 출력용
import time     # sleep 함수 사용

In [26]:
# labels에 id, pw가 리스트 형태로 저장
file_path = '/Users/hwang-gyubin/Downloads/030.방송콘텐츠 대화체 음성인식 데이터/01.데이터/Training/02.라벨링데이터/papago/key.txt'
labels = open(file_path).read().split(",")

client_id = labels[0] # 개발자센터에서 발급받은 Client ID 값
client_secret = labels[1] # 개발자센터에서 발급받은 Client Secret 값

In [27]:
def translate(text, lang='ko', target_lang='en') : 
    encText = urllib.parse.quote(text)
    data = f"source={lang}&target={target_lang}&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        result  = response_body.decode('utf-8')
        d = json.loads(result)
        print('--- translate --- ')
        print('번역전 : ', text)
        print('번역후 : ',d['message']['result']['translatedText'])
    else:
        print("Error Code:" + rescode)
        
    return d['message']['result']['translatedText']

In [14]:
translate("안녕하세요")


--- translate --- 
번역전 :  안녕하세요
번역후 :  Hello


'Hello'

In [34]:
# 데이터 로드
train_data_path = "/Users/hwang-gyubin/Downloads/030.방송콘텐츠 대화체 음성인식 데이터/01.데이터/Training/02.라벨링데이터/papago/mycsv.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,data,label
0,가진 거 다 내놔\n뭐래는거야 이 사람이\n지금 내말 안들으면 후회할텐데\n왜 그러...,1
1,철민아 오늘은 얼마 가지고 왔냐?\n오늘은 만원 밖에 없어요.\n진짜냐?\n정말이에...,1
2,얼마 가져왔어?\n십만원 정도.\n야 이걸로 새 폰을 어떻게 사\n이게 최선이었어....,1
3,핸드폰 좋아보이네 \n저요?\n핸드폰 보고 있는 사람 너잖아\n제 핸드폰인데 왜 그...,1
4,우와 이거 예쁘다! 나 주라.\n 싫어 내 거야.\n 싫어 예뻐서 내가 갖고 싶어....,1


In [26]:
# 루프 돌면서 번역기 ㄱㄱ
translates_en = []
for text in train_data['data'] :
    try:
        translates_en.append(translate(text))
    except ValueError:
        # Code to handle a ValueError
        print("Error: Invalid value")


--- translate --- 
번역전 :  가진 거 다 내놔
뭐래는거야 이 사람이
지금 내말 안들으면 후회할텐데
왜 그러세요 무섭게
와 많이도 들고 다니네 학생이 이렇게 돈이 많아?
제발요 그거 우리가족 한달 생활비에요
 너 지금 이거 안 내놓으면 우리한테 맞아 죽을지도 몰라 좋은말할때 꺼져
제발 돌려주세요.
번역후 :  Give me everything you have
What is he saying
If you don't listen to me now, you'll regret it
Why are you doing? in a frightful
Wow, you carry around a lot. A student has this much money?
Please. That's my family's monthly living expenses
 If you don't give it out now, we'll beat you to death. Get out of here when you say something nice
Please give it back.
--- translate --- 
번역전 :  철민아 오늘은 얼마 가지고 왔냐?
오늘은 만원 밖에 없어요.
진짜냐?
정말이에요.
거짓말이면 진짜 어떻게 되는줄 알지?
사실 만원 더 있어요.
진작에 그럴 것이지. 또 있는건 아니겠지?
진짜 없어요.
킥킥 쫄기는. 내일도 부탁해?
네.
번역후 :  Cheolmin, how much did you bring today?
We only have 10,000 won today.
Are you serious?
I'm serious.
You know what happens when you lie, right?
Actually, I have 10,000 won more.
I'm sure it's a long ago. Is there anything else?
I really don't have it's.
He's scared. Do you want me to do it a

HTTPError: HTTP Error 429: Too Many Requests

In [ ]:
# 루프 돌면서 번역기 ㄱㄱ
translates_ko = []
idx = 1;
for text in tqdm(translates_en) :
    try:
        ## 딜레이 없이 요청하면, 네이버측 서버 무리..? 차단 당할까바 
        if idx % 30 == 0 :
            print('-' * 30)
            now = datetime.datetime.now()
            print(now.strftime('%H:%M:%S')) # 현재 시간 출력

            time.sleep(7) # 7 초 딜레이

            now = datetime.datetime.now()
            print(now.strftime('%H:%M:%S'))

        translates_ko.append(translate(text, 'en', 'ko'))
    except ValueError:
        # Code to handle a ValueError
        print("Error: Invalid value")


In [32]:
len(translates_ko)


35

In [21]:
train_data['label'][idx]

1

In [35]:
clone_translates = []
for idx, text in enumerate(translates_ko) :
    try:
        ## 데이터 셋 : 인덱스, 클래스, 텍스트
        clone_translates.append([idx, train_data['label'][idx], text  ])
    except ValueError:
        # Code to handle a ValueError
        print("Error: Invalid value")

In [36]:
clone_translates

[[14,
  1,
  '이봐, 담배 있어\n아무도.나는 담배를 피우지 않는다\n담배 말고 현금은 없나요? 내가 담배를 사줄게\n나한테 왜 이러는 거야? 현금도 안 들고 다니는데\n때리겠어요? 그냥 나한테 줘.\n미안하지만, 아무것도 없어요\n뒤지고 나오면 죽여버릴 거야\n죄송해요. 제가 잘못했으니 나가주세요\n네가 가진 모든 것을 꺼내라\n좋아요.'],
 [15,
  1,
  '야! 저기 시계 좋아보여\n그렇지만\n뭐라고?너무 짧아요\n먼저 반말을 했는데 화가 났나요?\n그냥 그에게 좋은 시계를 주면 그는 그것으로 큰 일을 하고 있다\n내가 왜 너에게 시계를 줘야 해!\n한 대 맞자\n제발 그만 좀 해요\n내가 시계를 줄게요\n좋은 출발을 해드릴게요. 당신은 내 주먹을 사용하게 만든다'],
 [16,
  1,
  '당신은 사업을 하고 싶지 않나요?\n그 사람 왜 그래요.\n제가 자리를 빌려달라고 했었나요, 말랬나요?\n요즘 세상에 집세가 없어요. 나는 월세를 공평하게 내고 있다.\n여기가 우리 지역인 거 몰라?\n하. 제발 그만해.\n여기서 거래하려면 돈을 지불해야 한다\n계속 이렇게 하면 신고할게요\n먹어봐. 내가 너의 눈알을 뽑아줄게\n왜 나한테 그러는 거야.'],
 [17,
  1,
  '이봐, 네가 가진 모든 것을 내게 줘\n말도 안돼. 덤벼라\n당신은 나한테 잘 못해요. 너는 바보같아.\n살려주세요. 부탁합니다.\n당신은 나와 싸웠으니 돈을 더 줘야 합니다.\n실례지만, 잠시만 기다려 주시겠어요?\n난 널 내버려두지 않을 거야. 그의 주머니를 들여다봐\n100원입니다'],
 [18,
  1,
  '아, 처음 보는 시계네요?\n아, 나 이거 최근에 샀어\n일주일만 빌려주실 수 있나요?\n아니요, 비싸요\n지금 바로 돌려드리겠습니다\n원하지 않아요.\n그가 좋은 말을 하면 빌려주세요\n별로인데요.\n빌려줄 수 있어요? 나한테 뺏겨서?\n제가 빌려드릴게요.'],
 [19,
  1,
  '담배 있습니까?\n전 아무것도 없어요.\n그럼 저기 편의점에서 하나 

In [37]:
# 배열 to csv
fields = ['idx', 'class', 'conversation']
with open('translate.csv', 'w',newline='') as f: 
      
    # using csv.writer method from CSV package 
    write = csv.writer(f) 
      
    write.writerow(fields) 
    write.writerows(clone_translates)

In [ ]:
#https://github.com/naver/naver-openapi-guide/blob/draft/sample/python/APIExamTranslateNMT.py